In [4]:
# import dash
# import dash_core_components as dcc
# import dash_html_components as html
# import dash_bootstrap_components as dbc
# from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
# import dash_table
import json
import joblib
import sys
PATH_PI = 'C:/Repo/MiM_Analytics_Tesis/Tesis/dash_app/'
sys.path.insert(0, PATH_PI)
from ClassAppPreprocessing import ClassPreprocessing

C:\Users\fabri\AppData\Roaming\Python\Python38\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.0, the latest is 0.5.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [21]:
# Load data
config_data = json.load(open(PATH_PI+'config_dash.json'))
preprocessing = ClassPreprocessing(config_data)
df_season_players = preprocessing.load_score_process_data(dataset='player_seasons', source='csv', generate_dropdown=False)
df_players = preprocessing.load_score_process_data(dataset='player_all_seasons', source='csv')
dropdown_options = joblib.load(PATH_PI+'data/dropdown_options.pkl')
countries_latam = ['Argentina', 'Brazil', 'Chile', 'Colombia', 'Mexico', 'Peru']
dropdown_options = {country: dropdown_options[country] for country in countries_latam}

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator RobustScaler from version 1.0.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator RobustScaler from version 1.0.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your

In [25]:
dict_perf_index = joblib.load('C:/Repo/MiM_Analytics_Tesis/Tesis/performance_index/'+"20220210_PerformanceIndexObject.pkl")

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator RobustScaler from version 1.0.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [26]:
dict_perf_index

{'cols': {'player_cols': ['league_season',
   'team_id',
   'team_name',
   'player_id',
   'player_name',
   'player_preferred_position',
   'player_minutes',
   'wavg_player_rating'],
  'F': ['np_goals_p90',
   'shots_p90',
   'shooting_accuracy',
   'goal_conversion_np',
   'passing_accuracy',
   'assists_p90',
   'key_passes_p90',
   'dribbles_p90',
   'dribbles_success_ratio',
   'tackles_p90',
   'interceptions_p90'],
  'M': ['passes_p90',
   'passing_accuracy',
   'key_passes_p90',
   'scoring_contribution',
   'dribbles_p90',
   'dribbles_success_ratio',
   'fouls_drawn_p90',
   'fouls_committed_p90',
   'dribbles_past_p90',
   'tackles_p90',
   'interceptions_p90'],
  'D': ['passes_p90',
   'passing_accuracy',
   'fouls_drawn_p90',
   'fouls_committed_p90',
   'dribbles_past_p90',
   'duels_p90',
   'duels_success_ratio',
   'tackles_p90',
   'blocks_p90',
   'interceptions_p90',
   'penalty_committed_p90'],
  'G': ['saves_p90',
   'goals_conceded_p90',
   'goals_conceded_rati

In [29]:
dropdown_options

{'Argentina': {'2016/17': {'Racing Club': array(['Lautaro Martinez', 'Lisandro Lopez', 'Pablo Maximiliano Cuadra',
          'Santiago Rosales', 'Gustavo Bou', 'Marcos Acuna',
          'Ezequiel Videla', 'Marcelo Meli', 'Luciano Aued',
          'Diego Gonzalez', 'Ivan Pillud', 'Gaston Diaz', 'Pablo Alvarez',
          'Marco Torsiglieri', 'Leandro Grimi', 'Miguel Barbieri',
          'Sergio Vittor', 'Emanuel Insua', 'Gaston Gomez', 'Agustin Orion'],
         dtype=object),
   'Velez Sarsfield': array(['Maximiliano Romero', 'Juan Manuel Martinez', 'Mariano Pavone',
          'Nicolas Delgadillo', 'Santiago Caseres', 'Fabricio Alvarenga',
          'Nicolas Dominguez', 'Matias Vargas', 'Diego Zabala',
          'Leandro Desabato', 'Jorge Correa', 'Hector Canteros',
          'Gonzalo Diaz', 'Maximiliano Caire', 'Brian Cufre',
          'Lautaro Gianetti', 'Fabian Cubero', 'Emiliano Amor',
          'Fausto Grillo', 'Cristian Nasuti', 'Fabian Assmann',
          'Alan Aguerre'], dtype=

In [30]:
selected_country='Chile'
season_options = [{'label': i, 'value': i} for i in dropdown_options['{}'.format(selected_country)].keys()]

In [31]:
season_options

[{'label': '2018/19', 'value': '2018/19'},
 {'label': '2019/20', 'value': '2019/20'},
 {'label': '2020/21', 'value': '2020/21'},
 {'label': '2021/22', 'value': '2021/22'}]

In [13]:

def create_player_table(df, phase, season):
    """
    Creates table of statistics to display for selected player in a specific phase of the game and season
    :param df: dataframe with player's metrics
    :param phase: phase of the game (attack, build-up, defense, general stats)
    :param season: season of the league
    :return:
        Filtered dataframe with relevant statistics to display
    """
    df_filtered = df[df['league_season'] == season][config_data["DashColumns"][phase]].transpose().reset_index()
    df_filtered.columns = ['Var', 'Value']
    for col in df_filtered['Var'].to_list():
        if col in config_data["PctVars"]:
            df_filtered.loc[df_filtered['Var'] == col, 'Value'] = df_filtered.loc[df_filtered['Var'] == col, 'Value'].apply(lambda x: str(round(x*100))+'%')
        elif col == 'player_preferred_position':
            pass
        else:
            df_filtered.loc[df_filtered['Var'] == col, 'Value'] = df_filtered.loc[df_filtered['Var'] == col, 'Value'].apply(lambda x: round(x*-1, 2) if x < 0 else round(x, 2))

    df_filtered['Feature'] = df_filtered['Var'].map(config_data['ColumnNames'])

    return df_filtered[['Feature', 'Value']]

In [16]:
player='Lautaro Martinez'
df_player = df_season_players.loc[(df_season_players['player_name'] == player), :]

In [19]:
df_filtered = df_player[df_player['league_season'] == config_data['Parameters']['current_season']][config_data["DashColumns"]['stats_season']].transpose().reset_index()

In [20]:
df_filtered

,index
0,player_preferred_position
1,player_preferred_number
2,player_minutes
3,wavg_player_rating
4,goals_total
5,goals_assists
